In [1]:
#데이터 준비
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt', 'shopping.txt')

('shopping.txt', <http.client.HTTPMessage at 0x24be8e67408>)

In [2]:
import pandas as pd
import numpy as np

raw = pd.read_table("shopping.txt",names=['rating','review'])
print(raw)

#정답 기준 악플 0 정상적인글 1 rating값이 3보다 크면 1 작으면 0
raw['label'] = np.where( raw['rating'] >3 ,1 ,0 )



        rating                                             review
0            5                                            배공빠르고 굿
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2            5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4            5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...        ...                                                ...
199995       2                                    장마라그런가!!! 달지않아요
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998       5                                      넘이쁘고 쎄련되보이네요~
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다

[200000 rows x 2 columns]


In [3]:
#데이터전처리
#오타,띄어쓰기를 제거 
#단어 or 문자로 숫자로 치환  
# 단어로할경우 유니크한 단어만 수십만개가 나오는 문제점

#오타,띄어쓰기를 제거 
#한글에서 표현할 수있는 모든 단어들(오타포함) ->오타제거하고싶을 때 소프트웨어 사용(영어도포함시켜봄)
raw['review'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣a-z ]',"")  #이글자가아닌것은 ""공백으로 제거
print(raw)

#공백처리 제거
print(raw.isnull().sum())

#중복 제거
raw.drop_duplicates( subset=['review'], inplace=True)
print(raw)

#bag of words 문자->정수로 바꾸기
#review칼럼 데이터를 리스트로 만들기
uniqueString = raw['review'].tolist()
uniqueString = ''.join(uniqueString)
print(uniqueString[0:10]) 
#유니크한 문자만 담긴 list
uniqueString = list(set(uniqueString))
uniqueString.sort()
# print(uniqueString([0:100])


c:\Users\user\anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


        rating                                             review  label
0            5                                            배공빠르고 굿      1
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0
2            5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...      1
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...      0
4            5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1
...        ...                                                ...    ...
199995       2                                    장마라그런가!!! 달지않아요      0
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...      1
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요      1
199998       5                                      넘이쁘고 쎄련되보이네요~      1
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다      1

[200000 rows x 3 columns]
rating    0
review    0
label     0
dtype: int64
        rating                                  

In [13]:
import tensorflow as tf
#텐서플로우 라이브러리에서 자동적으로 문자에서 정수로 바꿔주는 기능이있음: 토큰나이징
from tensorflow.keras.preprocessing.text import Tokenizer

#글자단위를 정수로 변환 false할시 단어단위를 정수로 
#oov_token은 신조어를 정수로 치환하기위한 용도
tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')

#토크나이징하고싶은 리스트를 적으면됌
#참고사항단어단위로 오타전처리할경우에는  ex (안뎡:1, 반가워:2) 주로 삭제한다고함
StringList = raw['review'].tolist()
tokenizer.fit_on_texts(StringList)
word_index = tokenizer.word_index
print(tokenizer.word_index)
print(StringList[0:10])   

#train 데이터셋 전체 정수로 변환하기
train_seq=tokenizer.texts_to_sequences(StringList) #text리스트들을 각 정수를 나태줌
print(train_seq[0:10])

#정답확인
#최대글자를 동일하게 개수를 만들어야함
Y = raw['label'].tolist()
print(Y[0:10])



{'<OOV>': 1, ' ': 2, '요': 3, '.': 4, '이': 5, '고': 6, '다': 7, '아': 8, '는': 9, '어': 10, '하': 11, '니': 12, '가': 13, '서': 14, '도': 15, '지': 16, '네': 17, '에': 18, '데': 19, '좋': 20, '구': 21, '사': 22, '기': 23, '해': 24, '은': 25, '로': 26, '만': 27, '나': 28, '무': 29, '보': 30, '있': 31, '매': 32, '게': 33, '~': 34, '리': 35, '그': 36, '잘': 37, '안': 38, '한': 39, '주': 40, '라': 41, '배': 42, '거': 43, '너': 44, '했': 45, '습': 46, '시': 47, '송': 48, '용': 49, '품': 50, '으': 51, '제': 52, '!': 53, '같': 54, '을': 55, '입': 56, '상': 57, 'ㅠ': 58, '들': 59, '인': 60, '자': 61, '면': 62, '재': 63, '스': 64, '정': 65, '먹': 66, '비': 67, '합': 68, '부': 69, '대': 70, '없': 71, '여': 72, '려': 73, '마': 74, '음': 75, '맛': 76, '^': 77, '되': 78, '전': 79, '더': 80, '장': 81, '쓰': 82, '문': 83, '일': 84, '것': 85, 'ㅎ': 86, '건': 87, '저': 88, '않': 89, '수': 90, '생': 91, '르': 92, '빠': 93, '세': 94, '분': 95, '편': 96, '진': 97, '조': 98, '오': 99, '감': 100, '았': 101, '물': 102, '번': 103, '드': 104, '었': 105, '할': 106, '개': 107, '많': 108, '각': 109, '냥': 110, '겠'

AttributeError: module 'numpy' has no attribute 'arry'

In [5]:
#리뷰문자변환후 길이로 변환
raw['length'] = raw['review'].str.len()

#맨위에 데이터들만 보여줌
print(raw.head())
#통계
print(raw.describe())
#label은 반반해야 악플인지 판단하기 좋음

#최대 길이를 100개수미만 출력
raw['length'][raw['length'] < 100].count()



   rating                                             review  label  length
0       5                                            배공빠르고 굿      1       7
1       2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0      29
2       5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...      1      68
3       2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...      0     136
4       5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1      33
              rating          label         length
count  199908.000000  199908.000000  199908.000000
mean        3.225909       0.499995      39.699402
std         1.645431       0.500001      29.168119
min         1.000000       0.000000       3.000000
25%         2.000000       0.000000      17.000000
50%         2.000000       0.000000      29.000000
75%         5.000000       1.000000      55.000000
max         5.000000       1.000000     140.000000


189043

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#모든 문장 길이를 100자로 제한 이상 이하시 알아서 추가 제거함
X = pad_sequences(train_seq, maxlen = 100)

#train/test/val쪼개기 
#random_state는 seed
from sklearn.model_selection import train_test_split
Y=np.array(Y)
trainX, valX, trainY, valY = train_test_split( X, Y, test_size=0.2, random_state=42)

print(len(trainX))
print(len(valX))


159926
39982


In [18]:
import tensorflow as tf

# 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(word_index)+1, 16),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(1, activation='sigmoid'),  # 31개의 클래스를 예측
])


# 모델 컴파일. 여기서 'sparse_categorical_crossentropy'를 사용
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

# 모델 학습. Y는 정수 레이블 데이터입니다.
model.fit(X, Y, batch_size=64, epochs=30, verbose=2)


Epoch 1/30
3124/3124 - 306s - loss: 0.3049 - accuracy: 0.8809 - 306s/epoch - 98ms/step
Epoch 2/30
3124/3124 - 313s - loss: 0.2463 - accuracy: 0.9082 - 313s/epoch - 100ms/step
Epoch 3/30
3124/3124 - 329s - loss: 0.2281 - accuracy: 0.9158 - 329s/epoch - 105ms/step
Epoch 4/30
3124/3124 - 325s - loss: 0.2170 - accuracy: 0.9200 - 325s/epoch - 104ms/step
Epoch 5/30
3124/3124 - 314s - loss: 0.2076 - accuracy: 0.9238 - 314s/epoch - 100ms/step
Epoch 6/30
3124/3124 - 353s - loss: 0.1991 - accuracy: 0.9278 - 353s/epoch - 113ms/step
Epoch 7/30
3124/3124 - 357s - loss: 0.1923 - accuracy: 0.9305 - 357s/epoch - 114ms/step
Epoch 8/30
3124/3124 - 349s - loss: 0.1859 - accuracy: 0.9327 - 349s/epoch - 112ms/step
Epoch 9/30
3124/3124 - 355s - loss: 0.1801 - accuracy: 0.9353 - 355s/epoch - 114ms/step
Epoch 10/30
3124/3124 - 383s - loss: 0.1743 - accuracy: 0.9377 - 383s/epoch - 122ms/step
Epoch 11/30
3124/3124 - 351s - loss: 0.1683 - accuracy: 0.9399 - 351s/epoch - 112ms/step
Epoch 12/30
3124/3124 - 315s - 

In [4]:
from konlpy.tag import Okt
okt = Okt()
text = "아버지가 방에 들어가신다."
tokens = okt.morphs(text)
print(tokens)

['아버지', '가', '방', '에', '들어가신다', '.']
